# Denoising autoencoder

## 1. Notebook setup

### 1.1. Imports

In [ ]:
# Standard library
import sys
from pathlib import Path

# Third-party
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

# Add src directory to path
sys.path.append(str(Path.cwd().parent))

# Local imports
from src.data_utils import load_coco_cached
from src.visualization import plot_image_grid
from src.metrics import calculate_psnr, calculate_ssim
from src.noise import add_gaussian_noise, add_salt_pepper_noise, add_speckle_noise


### 1.2. Configuration

In [ ]:
# Training configuration
TRAIN_MODEL = True  # Set to False to download pre-trained model from HuggingFace

# Training hyperparameters
LATENT_DIM = 128
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# Noise parameters
NOISE_FACTOR = 0.25

# GPU configuration
GPU_ID = 1  # Which GPU to use (0-indexed). Set to None to use all available GPUs.

# Create models directory
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Configure GPU
GPU_ID = 0  # Which GPU to use (0-indexed). Set to None to use all available GPUs.
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        if GPU_ID is not None:

            # Use specific GPU
            tf.config.set_visible_devices(gpus[GPU_ID], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[GPU_ID], True)
            print(f'Using GPU {GPU_ID}: {gpus[GPU_ID].name}')

        else:

            # Use all GPUs with memory growth
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)

            print(f'Using {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}')

    except RuntimeError as e:
        print(e)
else:
    print('No GPU available, using CPU')

Using GPU 1: /physical_device:GPU:1


## 2. Data preparation

We'll use a 10% subset of the COCO 2017 dataset (~11,800 diverse images across 80 categories).

In [3]:
# Load COCO dataset (10% subset with caching)
# First run: Downloads full COCO (~95GB) and saves 10% subset to ../data/
# Subsequent runs: Loads quickly from cached subset file
(x_train, y_train), (x_test, y_test) = load_coco_cached(subset_percent=10, normalize=True)

print(f'Training set: {x_train.shape}')
print(f'Test set: {x_test.shape}')
print(f'Value range: [{x_train.min():.2f}, {x_train.max():.2f}]')

Training set: (11829, 64, 64, 3)
Test set: (1000, 64, 64, 3)
Value range: [0.00, 1.00]


## Visualize Different Noise Types

In [5]:
# Select a sample image
sample_idx = 42
sample_image = x_train[sample_idx:sample_idx+1]

# Create noisy versions
gaussian_noisy = add_gaussian_noise(sample_image, noise_factor=0.3)
salt_pepper_noisy = add_salt_pepper_noise(sample_image, amount=0.05)
speckle_noisy = add_speckle_noise(sample_image, noise_factor=0.2)

# Plot comparison
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(sample_image[0])
axes[0].set_title('Original', fontsize=14)
axes[0].axis('off')

axes[1].imshow(gaussian_noisy[0])
axes[1].set_title('Gaussian Noise', fontsize=14)
axes[1].axis('off')

axes[2].imshow(salt_pepper_noisy[0])
axes[2].set_title('Salt & Pepper Noise', fontsize=14)
axes[2].axis('off')

axes[3].imshow(speckle_noisy[0])
axes[3].set_title('Speckle Noise', fontsize=14)
axes[3].axis('off')

plt.tight_layout()
plt.show()

ValueError: high <= 0

## Create Training Data with Noise

For this example, we'll focus on Gaussian noise as it's the most common type in real-world scenarios.

In [ ]:
# Create noisy training and test sets
noise_factor = 0.25

x_train_noisy = add_gaussian_noise(x_train, noise_factor=noise_factor)
x_test_noisy = add_gaussian_noise(x_test, noise_factor=noise_factor)

print(f'Created noisy datasets with noise factor: {noise_factor}')
print(f'Noisy training set: {x_train_noisy.shape}')
print(f'Noisy test set: {x_test_noisy.shape}')

In [ ]:
# Visualize clean vs noisy samples
n_samples = 8
indices = np.random.choice(len(x_train), n_samples, replace=False)

fig, axes = plt.subplots(2, n_samples, figsize=(20, 5))
fig.suptitle('Clean Images (top) vs Noisy Images (bottom)', fontsize=16, y=1.02)

for i, idx in enumerate(indices):
    axes[0, i].imshow(x_train[idx])
    axes[0, i].axis('off')
    
    axes[1, i].imshow(x_train_noisy[idx])
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## Build Denoising Autoencoder

The architecture is similar to a compression autoencoder, but:
- **Input**: Noisy images
- **Output**: Clean images
- The bottleneck forces the network to learn noise-resistant features

In [ ]:
def build_denoising_autoencoder(input_shape=(64, 64, 3), latent_dim=128):
    '''
    Build a denoising autoencoder.
    
    Args:
        input_shape: Shape of input images
        latent_dim: Dimension of latent bottleneck
    
    Returns:
        Compiled Keras model
    '''
    # Encoder
    encoder_input = keras.Input(shape=input_shape)
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(encoder_input)
    x = layers.MaxPooling2D(2, padding='same')(x)  # 32x32
    
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)  # 16x16
    
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)  # 8x8
    
    x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)  # 4x4
    
    # Bottleneck
    x = layers.Flatten()(x)
    latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)
    
    # Decoder
    x = layers.Dense(4 * 4 * 512, activation='relu')(latent)
    x = layers.Reshape((4, 4, 512))(x)
    
    x = layers.Conv2DTranspose(512, 3, activation='relu', strides=2, padding='same')(x)  # 8x8
    x = layers.Conv2DTranspose(256, 3, activation='relu', strides=2, padding='same')(x)  # 16x16
    x = layers.Conv2DTranspose(128, 3, activation='relu', strides=2, padding='same')(x)  # 32x32
    x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)   # 64x64
    
    decoder_output = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)
    
    # Build model
    autoencoder = keras.Model(encoder_input, decoder_output, name='denoising_autoencoder')
    
    # Compile
    autoencoder.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return autoencoder

In [ ]:
# Build model
model = build_denoising_autoencoder(latent_dim=128)
model.summary()

## Train the Denoising Autoencoder

Key insight: We train on **noisy inputs** but optimize to match **clean outputs**.

In [ ]:
# Train the model
# Input: noisy images, Target: clean images
history = model.fit(
    x_train_noisy,  # Noisy input
    x_train,        # Clean target
    validation_data=(x_test_noisy, x_test),
    epochs=20,
    batch_size=32,
    shuffle=True,
    verbose=1
)

## Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('MSE Loss', fontsize=12)
axes[0].set_title('Training Loss', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].set_title('Mean Absolute Error', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluate Denoising Performance

In [ ]:
# Denoise test images
x_test_denoised = model.predict(x_test_noisy, verbose=0)

# Calculate metrics
mse_noisy = np.mean((x_test - x_test_noisy) ** 2)
mse_denoised = np.mean((x_test - x_test_denoised) ** 2)

psnr_noisy = calculate_psnr(x_test, x_test_noisy)
psnr_denoised = calculate_psnr(x_test, x_test_denoised)

ssim_noisy = calculate_ssim(x_test, x_test_noisy)
ssim_denoised = calculate_ssim(x_test, x_test_denoised)

print('Performance Metrics:')
print('=' * 50)
print(f'\nNoisy Images vs Clean:')
print(f'  MSE:  {mse_noisy:.6f}')
print(f'  PSNR: {psnr_noisy:.2f} dB')
print(f'  SSIM: {ssim_noisy:.4f}')

print(f'\nDenoised Images vs Clean:')
print(f'  MSE:  {mse_denoised:.6f}')
print(f'  PSNR: {psnr_denoised:.2f} dB')
print(f'  SSIM: {ssim_denoised:.4f}')

print(f'\nImprovement:')
print(f'  MSE:  {(mse_noisy - mse_denoised) / mse_noisy * 100:.1f}% reduction')
print(f'  PSNR: {psnr_denoised - psnr_noisy:.2f} dB increase')
print(f'  SSIM: {(ssim_denoised - ssim_noisy) / (1 - ssim_noisy) * 100:.1f}% closer to perfect')

## Visual Comparison

In [ ]:
# Visualize denoising results
n_samples = 8
indices = np.random.choice(len(x_test), n_samples, replace=False)

fig, axes = plt.subplots(3, n_samples, figsize=(20, 7.5))
fig.suptitle('Denoising Results: Original (top) vs Noisy (middle) vs Denoised (bottom)', 
             fontsize=16, y=0.995)

for i, idx in enumerate(indices):
    # Original clean image
    axes[0, i].imshow(x_test[idx])
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Original', fontsize=12, rotation=0, labelpad=40, va='center')
    
    # Noisy image
    axes[1, i].imshow(x_test_noisy[idx])
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_ylabel('Noisy', fontsize=12, rotation=0, labelpad=40, va='center')
    
    # Denoised image
    axes[2, i].imshow(x_test_denoised[idx])
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_ylabel('Denoised', fontsize=12, rotation=0, labelpad=40, va='center')
    
    # Add metrics for each image
    psnr_val = calculate_psnr(x_test[idx:idx+1], x_test_denoised[idx:idx+1])
    ssim_val = calculate_ssim(x_test[idx:idx+1], x_test_denoised[idx:idx+1])
    axes[2, i].set_title(f'PSNR: {psnr_val:.1f}dB\nSSIM: {ssim_val:.3f}', 
                         fontsize=9)

plt.tight_layout()
plt.show()

## Experiment: Different Noise Levels

Let's test how the model performs on different noise levels.

In [ ]:
# Test on different noise levels
noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
results = []

for noise_factor in noise_levels:
    # Create noisy images
    x_test_noisy_temp = add_gaussian_noise(x_test, noise_factor=noise_factor)
    
    # Denoise
    x_test_denoised_temp = model.predict(x_test_noisy_temp, verbose=0)
    
    # Calculate metrics
    psnr_before = calculate_psnr(x_test, x_test_noisy_temp)
    psnr_after = calculate_psnr(x_test, x_test_denoised_temp)
    ssim_before = calculate_ssim(x_test, x_test_noisy_temp)
    ssim_after = calculate_ssim(x_test, x_test_denoised_temp)
    
    results.append({
        'noise_factor': noise_factor,
        'psnr_before': psnr_before,
        'psnr_after': psnr_after,
        'ssim_before': ssim_before,
        'ssim_after': ssim_after
    })

# Print results
print('\nPerformance across different noise levels:')
print('=' * 80)
print(f'{"Noise":>8} | {"PSNR Before":>12} | {"PSNR After":>12} | {"SSIM Before":>12} | {"SSIM After":>12}')
print('-' * 80)
for r in results:
    print(f'{r["noise_factor"]:>8.2f} | {r["psnr_before"]:>12.2f} | {r["psnr_after"]:>12.2f} | '
          f'{r["ssim_before"]:>12.4f} | {r["ssim_after"]:>12.4f}')

In [ ]:
# Plot performance across noise levels
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

noise_factors = [r['noise_factor'] for r in results]

# PSNR
axes[0].plot(noise_factors, [r['psnr_before'] for r in results], 
             'o-', label='Before Denoising', linewidth=2, markersize=8)
axes[0].plot(noise_factors, [r['psnr_after'] for r in results], 
             's-', label='After Denoising', linewidth=2, markersize=8)
axes[0].set_xlabel('Noise Factor', fontsize=12)
axes[0].set_ylabel('PSNR (dB)', fontsize=12)
axes[0].set_title('PSNR vs Noise Level', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# SSIM
axes[1].plot(noise_factors, [r['ssim_before'] for r in results], 
             'o-', label='Before Denoising', linewidth=2, markersize=8)
axes[1].plot(noise_factors, [r['ssim_after'] for r in results], 
             's-', label='After Denoising', linewidth=2, markersize=8)
axes[1].set_xlabel('Noise Factor', fontsize=12)
axes[1].set_ylabel('SSIM', fontsize=12)
axes[1].set_title('SSIM vs Noise Level', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Save the Model

In [ ]:
# Save the denoising model
model_path = '../models/denoising_ae_latent128.keras'
model.save(model_path)
print(f'Model saved to: {model_path}')

## Key Takeaways

1. **Training Strategy**: Denoising autoencoders are trained with noisy inputs and clean targets

2. **Robust Features**: The bottleneck forces the network to learn features that are resistant to noise

3. **Generalization**: The model trained on one noise level can still denoise images with different noise levels, though performance degrades with very high noise

4. **Applications**: Beyond image denoising, this approach is useful for:
   - Pretraining encoders for other tasks
   - Data augmentation
   - Regularization technique

5. **Limitations**: 
   - Works best when test noise is similar to training noise
   - May blur fine details while removing noise
   - Performance depends on bottleneck size and architecture